In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def huber_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    epsilon=1.35,
    alpha=0.0001,
    max_iter=1000,
    use_cv=False,
    cv_params=None,
    random_state=None
):
    """
    Robust Huber regression for time series forecasting with:
    - Automatic hyperparameter tuning via grid search
    - Outlier-resistant modeling
    - Enhanced differencing checks
    """
    # Create directory structure
    os.makedirs('HuberRegression', exist_ok=True)
    
    # 1. Stationarity and Differencing (fixed adf_result check)
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:  # Fixed p-value access
            d = 1
            series = series.diff().dropna()

    # 2. Feature Engineering - Lag features with type consistency
    df = pd.DataFrame({'y': series.astype(np.float64)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(np.float64)
    df = df.dropna()

    # 3. Time-based split with validation
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 4. Model training with optional CV
    if use_cv:
        # Default parameter grid
        param_grid = cv_params or {
            'epsilon': [1.0, 1.35, 1.5, 2.0],
            'alpha': [0.0001, 0.001, 0.01, 0.1]
        }
        
        grid_search = GridSearchCV(
            HuberRegressor(max_iter=max_iter),
            param_grid,
            cv=5,
            scoring='neg_mean_squared_error'
        )
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
        best_epsilon = model.epsilon
        best_alpha = model.alpha
    else:
        model = HuberRegressor(
            epsilon=epsilon,
            alpha=alpha,
            max_iter=max_iter
        )
        model.fit(X_train, y_train)
        best_epsilon = epsilon
        best_alpha = alpha

    # 5. Forecasting with type consistency
    pred_test = model.predict(X_test).astype(np.float64)

    # 6. Inverse differencing with edge case handling
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]

    # 7. Enhanced DataFrame with confidence intervals
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast': pred_test,
        'residual': y_test.values - pred_test
    })

    # 8. Robust CSV saving
    forecast_csv_path = os.path.join('HuberRegression', 'huber_forecasts.csv')
    forecast_df.to_csv(
        forecast_csv_path,
        mode='a',
        header=not os.path.exists(forecast_csv_path),
        index=False
    )
     # 9. Comprehensive metrics
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast']))
    metrics_data = {
        'district': district_name,
        'rmse': rmse,
        'r_squared': model.score(X_train, y_train),
        'coefficients': model.coef_.tolist(),
        'intercept': model.intercept_,
        'final_epsilon': best_epsilon,
        'final_alpha': best_alpha,
        'differencing': d,
        'n_iterations': model.n_iter_
    }

    if use_cv:
        metrics_data.update({
            'best_params': grid_search.best_params_,
            'cv_results': grid_search.cv_results_
        })

    metrics_df = pd.DataFrame([metrics_data])
    
    metrics_csv_path = os.path.join('HuberRegression', 'huber_metrics.csv')
    metrics_df.to_csv(
        metrics_csv_path,
        mode='a',
        header=not os.path.exists(metrics_csv_path),
        index=False
    )

    # 10. Enhanced plotting with residuals
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    
    # Forecast plot
    ax1.plot(original_series.index, original_series.values, label='Original Series')
    ax1.plot(forecast_df['date'], forecast_df['forecast'], 
            label='Huber Forecast', color='darkorange')
    ax1.set_title(f'Huber Regression Forecast for {district_name}\nRMSE: {rmse:.2f}')
    ax1.legend()

    # Residual plot
    ax2.bar(forecast_df['date'], forecast_df['residual'], 
           color='gray', alpha=0.7, width=5)
    ax2.axhline(0, color='red', linestyle='--')
    ax2.set_title('Forecast Residuals')
    
    plt.tight_layout()
    plt.savefig(os.path.join('HuberRegression', f'huber_forecast_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'model_params': model.get_params(),
        'feature_names': X_train.columns.tolist()
    }

In [4]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I48"].asfreq('MS')
        
        results = huber_regression_lags_only(
            ts,
            district,
            use_cv=True,
            cv_params={
                'epsilon': [1.0, 1.35, 1.7],
                'alpha': [0.0001, 0.001, 0.01]
            },
            max_iter=2000
        )
        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

=== Metrics ===
     district       rmse  r_squared  \
0  AHMEDNAGAR  81.647353   0.255549   

                                        coefficients   intercept  \
0  [0.5212986986910696, -0.0065529060430328185, 0...  213.483014   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.7       0.0001             0            85   

                         best_params  \
0  {'alpha': 0.0001, 'epsilon': 1.7}   

                                          cv_results  
0  {'mean_fit_time': [0.03844714164733887, 0.0168...  

=== Forecast Data ===
     district       date  actual    forecast   residual
0  AHMEDNAGAR 2020-07-01   354.0  416.810678 -62.810678
1  AHMEDNAGAR 2020-08-01   391.0  402.348872 -11.348872
2  AHMEDNAGAR 2020-09-01   417.0  423.708291  -6.708291
3  AHMEDNAGAR 2020-10-01   343.0  436.466125 -93.466125
4  AHMEDNAGAR 2020-11-01   468.0  398.572918  69.427082
=== Metrics ===
  district       rmse  r_squared  \
0    AKOLA  69.864608   -0.04815   

      

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
   district       rmse  r_squared  \
0  AMRAVATI  440.57789   0.235997   

                                        coefficients  intercept  \
0  [-0.36576789764287654, 0.14679641685645003, 0.... -27.647359   

   final_epsilon  final_alpha  differencing  n_iterations  \
0           1.35        0.001             1            54   

                         best_params  \
0  {'alpha': 0.001, 'epsilon': 1.35}   

                                          cv_results  
0  {'mean_fit_time': [0.017149877548217774, 0.011...  

=== Forecast Data ===
   district       date  actual    forecast    residual
0  AMRAVATI 2020-07-01     470  324.604990  145.395010
1  AMRAVATI 2020-08-01     482  265.119335  216.880665
2  AMRAVATI 2020-09-01     586  248.021804  337.978196
3  AMRAVATI 2020-10-01     819  188.954355  630.045645
4  AMRAVATI 2020-11-01     609   91.997666  517.002334
=== Metrics ===
     district        rmse  r_squared  \
0  AURANGABAD  308.153376  -0.001885   

          

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
     district        rmse  r_squared  \
0  CHANDRAPUR  275.219855   0.014591   

                                        coefficients  intercept  \
0  [-0.18267435640932037, -0.11178176438646624, -... -12.353763   

   final_epsilon  final_alpha  differencing  n_iterations  \
0           1.35       0.0001             1            49   

                          best_params  \
0  {'alpha': 0.0001, 'epsilon': 1.35}   

                                          cv_results  
0  {'mean_fit_time': [0.014331626892089843, 0.007...  

=== Forecast Data ===
     district       date  actual    forecast    residual
0  CHANDRAPUR 2020-07-01     445  327.138112  117.861888
1  CHANDRAPUR 2020-08-01     502  301.593239  200.406761
2  CHANDRAPUR 2020-09-01     592  266.643269  325.356731
3  CHANDRAPUR 2020-10-01     628  219.600804  408.399196
4  CHANDRAPUR 2020-11-01     562  184.873477  377.126523


...

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
  district      rmse  r_squared  \
0   MUMBAI  223.3828  -0.207647   

                                        coefficients  intercept  \
0  [0.017696076462086558, -0.1643420210547586, -0... -52.502633   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.0        0.001             1            42   

                        best_params  \
0  {'alpha': 0.001, 'epsilon': 1.0}   

                                          cv_results  
0  {'mean_fit_time': [0.010499286651611327, 0.006...  

=== Forecast Data ===
  district       date  actual    forecast    residual
0   MUMBAI 2020-07-01     513  958.937148 -445.937148
1   MUMBAI 2020-08-01     579  881.935526 -302.935526
2   MUMBAI 2020-09-01     683  816.291601 -133.291601
3   MUMBAI 2020-10-01     763  811.521431  -48.521431
4   MUMBAI 2020-11-01     583  718.377910 -135.377910
=== Metrics ===
  district        rmse  r_squared  \
0   NAGPUR  181.999659   0.377546   

                            

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
  district       rmse  r_squared  \
0   WASHIM  21.639198   0.281089   

                                        coefficients  intercept  \
0  [0.43078041756007157, -0.029628198100675938, 0...  53.045698   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.7         0.01             0            52   

                       best_params  \
0  {'alpha': 0.01, 'epsilon': 1.7}   

                                          cv_results  
0  {'mean_fit_time': [0.014958906173706054, 0.008...  

=== Forecast Data ===
  district       date  actual    forecast   residual
0   WASHIM 2020-07-01   154.0  109.645275  44.354725
1   WASHIM 2020-08-01   127.0  133.976434  -6.976434
2   WASHIM 2020-09-01   120.0  118.517893   1.482107
3   WASHIM 2020-10-01   113.0  123.034296 -10.034296
4   WASHIM 2020-11-01   154.0  116.358966  37.641034
=== Metrics ===
   district       rmse  r_squared  \
0  YAVATMAL  25.049299   0.490251   

                                  

{'district': 'MUMBAI SUBURBAN',
 'forecast_df':           district       date  actual    forecast    residual
 0  MUMBAI SUBURBAN 2021-02-01   607.0  661.314964  -54.314964
 1  MUMBAI SUBURBAN 2021-03-01   618.0  777.363259 -159.363259,
 'metrics_df':           district        rmse  r_squared  \
 0  MUMBAI SUBURBAN  119.052013   0.412722   
 
                                         coefficients    intercept  \
 0  [0.2729698657633795, -0.3407554832475997, -0.5...  1284.866349   
 
    final_epsilon  final_alpha  differencing  n_iterations  \
 0            1.7         0.01             0            76   
 
                        best_params  \
 0  {'alpha': 0.01, 'epsilon': 1.7}   
 
                                           cv_results  
 0  {'mean_fit_time': [0.009065771102905273, 0.011...  ,
 'model_params': {'alpha': 0.01,
  'epsilon': 1.7,
  'fit_intercept': True,
  'max_iter': 2000,
  'tol': 1e-05,
  'warm_start': False},
 'feature_names': ['lag_1', 'lag_2', 'lag_3']}